In [3]:
import torch
import pandas as pd
import numpy as np
from pylab import mpl, plt
plt.style.use('seaborn')

from torch.utils.data.sampler import SequentialSampler
from torch.utils.data.dataloader import DataLoader
from sklearn.preprocessing import MinMaxScaler
# from torch.utils.data import Dataset


print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 2060


## Load Date

- `class ClosePriceDataset()`: `Dataset`
    - `__init__`: read_csv()
        - `MinMaxScaler`: [-1, 1]，(可以被reverse回原值)
- `def train_valid_split`: return `Sampler`, split training data(80%) and validation data(20%)
- `train_loader`: return `DataLoader`
- `val_loader`: return `DataLoader`

In [2]:

# Min-Max Normalization
scaler = MinMaxScaler(feature_range=(-1, 1))

class ClosePriceDataset(torch.utils.data.Dataset):
    """
    self.n_steps: int
    self.n_days, self.label: ndarray
    """
    def __init__(self, root, n_steps):
        super(ClosePriceDataset, self).__init__()
        data = pd.read_csv(root, index_col='Date')
        data = data[['Close']]
        # data.info()

        self.n_steps = n_steps
        # self.n_days = scaler.fit_transform(self.n_days.values.reshape(-1, 1))
        scaled_prices = scaler.fit_transform(data.values)
        scaled_prices = scaled_prices.reshape((-1,))
        # print(f'total days: {scaled_prices.shape}')
        self.n_days, self.label = self.split_sequence(scaled_prices, n_steps)

    def __getitem__(self, index):
        # print(f'n_days: {self.n_days[index]}, label: {self.label[index]}')
        return self.n_days[index], self.label[index]

    def __len__(self):
        return len(self.label)

    def split_sequence(slef, scaled_sequence, timestep):
        """
        return X: ndarray (*, 1, timestep)
        return y: ndarray (1,)
        """
        X = list()
        y = list()

        for i in range(len(scaled_sequence)):
            end_idx = i + timestep
            if end_idx > len(scaled_sequence) - 1:
                break
            seq_X, seq_y = [scaled_sequence[i:end_idx]], scaled_sequence[end_idx]
            X.append(seq_X)
            y.append(seq_y)

        
        X = np.array(X)
        y = np.array(y)
        # print out first record
        # print(X[0:2])
        # print(y[0:2])
        return torch.Tensor(X), torch.Tensor(y)


def train_valid_split(dataset, val_split = 0.2, random_seed = 0):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_idx = int(np.floor(dataset_size * val_split))
    train_indices, val_indices = indices[val_idx:], indices[:val_idx]
    train_sampler = SequentialSampler(train_indices)
    valid_sampler = SequentialSampler(val_indices)

    return train_sampler, valid_sampler


batch_size = 32
val_split = 0.2
random_seed= 42

dataset = ClosePriceDataset(root = 'datasets/BRK-A.csv', n_steps = 60)
train_sampler, valid_sampler = train_valid_split(dataset=dataset, val_split=val_split, random_seed=random_seed)
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=1, sampler=valid_sampler)

print("train size: ", len(train_sampler))
print("valid size: ", len(valid_sampler))
print(f'(training data) batch size: {batch_size}, #batches: {len(train_loader)}')

# dataiter = iter(train_loader)
# data = dataiter.next()
# features, labels = data
# print(scaler.inverse_transform(features.reshape(-1, 1)), scaler.inverse_transform(labels.reshape(1, -1)))
# print(features.shape, labels.shape)

# dataiter = iter(val_loader)
# data = dataiter.next()
# features, labels = data
# print(features.shape, labels.shape)
# print(scaler.inverse_transform(labels.reshape(1, -1)))

tensor([[[290., 290., 290., 290., 290.]],

        [[290., 290., 290., 290., 270.]],

        [[290., 290., 290., 270., 270.]],

        [[290., 290., 270., 270., 270.]],

        [[290., 270., 270., 270., 270.]],

        [[270., 270., 270., 270., 270.]],

        [[270., 270., 270., 270., 260.]],

        [[270., 270., 270., 260., 260.]],

        [[270., 270., 260., 260., 260.]],

        [[270., 260., 260., 260., 260.]],

        [[260., 260., 260., 260., 265.]],

        [[260., 260., 260., 265., 265.]],

        [[260., 260., 265., 265., 265.]],

        [[260., 265., 265., 265., 260.]],

        [[265., 265., 265., 260., 260.]],

        [[265., 265., 260., 260., 255.]],

        [[265., 260., 260., 255., 245.]],

        [[260., 260., 255., 245., 245.]],

        [[260., 255., 245., 245., 255.]],

        [[255., 245., 245., 255., 255.]],

        [[245., 245., 255., 255., 260.]],

        [[245., 255., 255., 260., 260.]],

        [[255., 255., 260., 260., 275.]],

        [[2

## Model

每一層需給定的input（計算上一層output）的size

input: (batch-size, channel, data-length)
(32, 1, 60)


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvNet(nn.Module):
    """Some Information about MyModule"""
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, groups=1, bias=True, kernel_size=20, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=16, groups=1, bias=True, kernel_size=5, stride=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(2 * 16, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # print('conv1 Shape: {}'.format(x.shape))
        x = self.maxpool1(x)
        # print('MaxPool1 Shape: {}'.format(x.shape))
        x = F.relu(self.conv2(x))
        # print('conv2 Shape: {}'.format(x.shape))
        x = self.maxpool2(x)
        # print('MaxPool2 Shape: {}'.format(x.shape))
        x = torch.flatten(x, start_dim=1)
        # print('Flaten Shape: {}'.format(x.shape))
        x = F.relu(self.fc1(x))
        # print('Linear1 Shape: {}'.format(x.shape))
        x = self.fc2(x)
        
        return x

net = ConvNet()
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

ConvNet(
  (conv1): Conv1d(1, 64, kernel_size=(2,), stride=(1,))
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)


## Training

丟進model需要為`Tensor`型態. 

inputs, labels: `Tensor`

In [4]:
epochs = 100
hist = np.zeros(epochs)
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        # inputs = torch.from_numpy(x)
        # labels = torch.from_numpy(y)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward() # calculate dl/dw
        optimizer.step() # update weights

        # print statistics
        running_loss += loss.item()
        # if (i % (len(train_loader) // 4) == 0) or (i == len(train_loader)-1):    # print loss every 5 mini-batches
        #   print(f'[ epoch: {epoch + 1}, batch: {i + 1:5d}] loss: {running_loss / 5:.3f}')
        #   running_loss = 0.0
    
    print(f'[ epoch: {epoch + 1} ].   MSE: {running_loss}')
    hist[epoch] = running_loss
print('Finished Training')


plt.plot(hist, label="Training loss")
plt.legend()
plt.show()

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,     1] loss: 23056.034


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([68])) that is different to the input size (torch.Size([68, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[2,     1] loss: 749.317
[3,     1] loss: 747.980
[4,     1] loss: 747.552
[5,     1] loss: 748.636
[6,     1] loss: 749.139
[7,     1] loss: 749.493
[8,     1] loss: 749.746
[9,     1] loss: 749.931
[10,     1] loss: 750.066
[11,     1] loss: 750.169
[12,     1] loss: 750.248
[13,     1] loss: 750.309
[14,     1] loss: 750.358
[15,     1] loss: 750.396
[16,     1] loss: 750.428
[17,     1] loss: 750.453
[18,     1] loss: 750.473
[19,     1] loss: 750.493
[20,     1] loss: 750.505
[21,     1] loss: 750.516
[22,     1] loss: 750.525
[23,     1] loss: 750.534
[24,     1] loss: 750.541
[25,     1] loss: 750.547
[26,     1] loss: 750.553
[27,     1] loss: 750.559


KeyboardInterrupt: 

## validation

計算validation data的RMSE

In [ ]:
import math

pred_value = np.zeros(len(val_loader))
true_value = np.zeros(len(val_loader))
loss = 0
for i, (inputs, labels) in enumerate(val_loader):
    pred = net(inputs)
    pred = scaler.inverse_transform(pred.detach().numpy())
    ans = scaler.inverse_transform(labels.reshape(-1, 1))
    pred_value[i] = pred
    true_value[i] = ans
    loss = (pred - ans) ** 2

print(loss/len(val_loader))


# Visualising the results
figure, axes = plt.subplots(figsize=(15, 6))
axes.xaxis_date()

axes.plot(df[len(df)-len(val_loader):].index, pred_value, color = 'red', label = 'Predicted Price')
axes.plot(df[len(df)-len(val_loader):].index, true_value, color = 'blue', label = 'Real Price')
#axes.xticks(np.arange(0,394,50))
plt.title('Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
# plt.savefig('ibm_pred.png')
plt.show()